<a href="https://colab.research.google.com/github/hyulianton/jejaringsosial/blob/master/js_model_SIS.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [18]:
# !pip install bokeh networkx

import networkx as nx
import numpy as np

from bokeh.plotting import figure, from_networkx, show
from bokeh.models import Circle, StaticLayoutProvider, Slider, CustomJS, Div, ColumnDataSource, Range1d
from bokeh.layouts import column, row
from bokeh.palettes import Category10

# Parameter dan inisialisasi graph
N = 50
p = 0.1
initial_infected_count = 5

G = nx.erdos_renyi_graph(N, p)

# Inisialisasi status node: 0=S, 1=I dengan jumlah I awal sesuai permintaan
status = np.zeros(N, dtype=int)
initial_infected = np.random.choice(N, size=initial_infected_count, replace=False)
status[initial_infected] = 1

# Fungsi update status satu langkah (model SIS)
beta = 0.3   # Probabilitas infeksi
gamma = 0.1  # Probabilitas sembuh (kembali S)

def update_status(G, status, beta, gamma):
    new_status = status.copy()
    for node in G.nodes():
        if status[node] == 0:
            neighbors = list(G.neighbors(node))
            infected_neighbors = sum(status[neighbors])
            if infected_neighbors > 0:
                if np.random.rand() < 1 - (1 - beta)**infected_neighbors:
                    new_status[node] = 1
        else:
            if np.random.rand() < gamma:
                new_status[node] = 0
    return new_status

# Simulasi selama beberapa langkah
num_steps = 20
status_list = [status.copy()]
for _ in range(num_steps):
    status = update_status(G, status, beta, gamma)
    status_list.append(status.copy())

# Layout agar node tidak terlalu bertumpuk dengan pengaturan jarak di spring_layout
pos = nx.spring_layout(G, k=1.3, iterations=50, seed=42)

# Visualisasi graph
plot_graph = figure(width=600, height=600,
                    x_range=Range1d(-1.5,1.5), y_range=Range1d(-1.5,1.5),
                    title="Model Penyebaran Penyakit SIS")

graph_renderer = from_networkx(G, pos)

color_map = {0: Category10[10][0], 1: Category10[10][1]}  # Biru=S, Merah=I
initial_colors = [color_map[st] for st in status_list[0]]

graph_renderer.node_renderer.data_source.data['fill_color'] = initial_colors
graph_renderer.node_renderer.glyph = Circle(radius=0.06, fill_color='fill_color')
graph_renderer.layout_provider = StaticLayoutProvider(graph_layout=pos)
plot_graph.renderers.append(graph_renderer)

source = graph_renderer.node_renderer.data_source
all_colors = [[color_map[st] for st in status] for status in status_list]

# Hitung jumlah node S dan I tiap langkah untuk info dan plot
counts_S = [np.sum(status == 0) for status in status_list]
counts_I = [np.sum(status == 1) for status in status_list]
steps = list(range(len(status_list)))

# Siapkan data sumber untuk plot jumlah S dan I
source_count = ColumnDataSource(data=dict(steps=steps, S=counts_S, I=counts_I))

# Plot jumlah S dan I waktu nyata
plot_counts = figure(width=600, height=300,
                     title="Jumlah Node Susceptible (S) dan Infected (I) Selama Simulasi",
                     x_axis_label='Langkah Waktu',
                     y_axis_label='Jumlah Node',
                     x_range=(0, num_steps), y_range=(0, N))

plot_counts.line('steps', 'S', source=source_count, line_width=2, color=Category10[10][0], legend_label="Susceptible (S)")
plot_counts.line('steps', 'I', source=source_count, line_width=2, color=Category10[10][1], legend_label="Infected (I)")
plot_counts.legend.location = "center"

# Widget informasi jumlah node saat ini
info_div = Div(text=f"<b>Langkah 0:</b> S = {counts_S[0]}, I = {counts_I[0]}", width=250, height=40)

# Slider untuk memilih langkah waktu
slider = Slider(start=0, end=len(status_list)-1, value=0, step=1, title="Langkah Waktu")

# Callback js untuk update warna node, teks, dan vertikal garis pada plot waktu
callback = CustomJS(args=dict(source=source, colors=all_colors, slider=slider,
                              info=info_div, source_count=source_count, plot_counts=plot_counts), code="""
    const step = slider.value;
    source.data['fill_color'] = colors[step];
    source.change.emit();
    info.text = `<b>Langkah ${step}:</b> S = ${source_count.data['S'][step]}, I = ${source_count.data['I'][step]}`;
""")

slider.js_on_change('value', callback)

# Tampilkan layout grafik dan slider
layout = column(plot_graph, slider, info_div, plot_counts)
show(layout)
